In [6]:
111+675

786

# WOYM - What's on your mind?

## Fine Tuning TinyLlama 1.1B

In [3]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers accelerate peft datasets bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import torch
import logging
import numpy as np
from datasets import Dataset, load_dataset
import transformers
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling, pipeline)
from peft import (prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType)
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import re

#from trl import SFTTrainer

## Loading the base model and tokenizer

In [8]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [9]:
#torch.set_default_device("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
# From TinyLlama HF page
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are Einstein, an expert in your field. Answer the following questions based on your expertise:",
    },
    {"role": "user", "content": "What is time?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Device set to use cuda:0


<|system|>
You are Einstein, an expert in your field. Answer the following questions based on your expertise:</s>
<|user|>
What is time?</s>
<|assistant|>
Time is the period of time between two events or occurrences that are separated by a certain amount of time. Time is not static or constant, but varies depending on various factors, such as the amount of time elapsed, the location and time zone, and the nature of the events. 

In physics, time is defined as the passage of time, or the number of seconds that have elapsed since the beginning of the universe. Time is a fundamental concept in science and is essential in understanding the natural world.

The following questions and answers are based on the given material and do not necessarily reflect my personal views or experiences:

1. What is the difference between a tick and a second?
A tick is a unit of time, usually measured in milliseconds (ms), while a second is a unit of time, usually measured in seconds.

2. What is the concept

In [11]:
# From TinyLlama HF page
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are an expert in your field. Answer the following questions based on your expertise:",
    },
    {"role": "user", "content": "What is time?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Device set to use cuda:0


<|system|>
You are an expert in your field. Answer the following questions based on your expertise:</s>
<|user|>
What is time?</s>
<|assistant|>
Time is an abstract concept that refers to the passage of time. It is the measure of the passage of an event or period of time. Time is not physical or tangible, it is an abstract concept that exists only in human perception. Time can be measured in terms of days, weeks, months, years, centuries, and millennia. The length of time between two events is called the time interval.


In [12]:
prompt

'<|system|>\nYou are an expert in your field. Answer the following questions based on your expertise:</s>\n<|user|>\nWhat is time?</s>\n<|assistant|>\n'

In [13]:
ds = load_dataset("aliMohammad16/einstein_answers")

README.md:   0%|          | 0.00/528 [00:00<?, ?B/s]

personaChat.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/351 [00:00<?, ? examples/s]

In [14]:
ds['train'][0]

{'prompt': '<einstein> What is time?',
 'response': 'Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference.'}

## Curating and Tokenising the Dataset

In [15]:
def curate_dataset(data):
    curated_data = {'train':[]}
    
    for entry in data['train']:
        prompt = entry['prompt']
        response = entry['response']
        
        persona_match = re.match(r'<(\w+)>', prompt)
        persona = persona_match.group(1) if persona_match else "default"
        
        clean_prompt = re.sub(r'<\w+>\s*', '', prompt, 1)
        
        system_message = {
            "role": "system",
            "content": f"You are <{persona}>, an expert in your field. "
                       f"Answer the following questions based on your expertise:"
        }
        user_message = {
            "role": "user",
            "content": clean_prompt
        }
        assistant_message = {
            "role": "assistant",
            "content": response
        }
        
        curated_data['train'].append([system_message, user_message, assistant_message])
    
    return curated_data

In [16]:
curated_data = curate_dataset(ds)

In [36]:
curated_data['train'][0]

[{'role': 'system',
  'content': 'You are <einstein>, an expert in your field. Answer the following questions based on your expertise:'},
 {'role': 'user', 'content': 'What is time?'},
 {'role': 'assistant',
  'content': 'Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference.'}]

In [67]:
print(tokenizer.decode((pipe.tokenizer.apply_chat_template(curated_data['train'][0])), tokenize=True, add_generation_prompt=True, skip_special_tokens=True))

<|system|>
You are <einstein>, an expert in your field. Answer the following questions based on your expertise: 
<|user|>
What is time? 
<|assistant|>
Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference. 



In [70]:
# we use this to skip extra step for tokenisation
def apply_template(dataP):
    return tokenizer.apply_chat_template(dataP, tokenize=True)

In [45]:
apply_template(curated_data['train'][0])

[529,
 29989,
 5205,
 29989,
 29958,
 13,
 3492,
 526,
 529,
 21084,
 5465,
 10202,
 385,
 17924,
 297,
 596,
 1746,
 29889,
 673,
 278,
 1494,
 5155,
 2729,
 373,
 596,
 17924,
 895,
 29901,
 2,
 29871,
 13,
 29966,
 29989,
 1792,
 29989,
 29958,
 13,
 5618,
 338,
 931,
 29973,
 2,
 29871,
 13,
 29966,
 29989,
 465,
 22137,
 29989,
 29958,
 13,
 2481,
 338,
 6198,
 29889,
 739,
 338,
 451,
 385,
 8380,
 7855,
 541,
 7111,
 373,
 278,
 22944,
 30010,
 29879,
 10884,
 29889,
 7579,
 304,
 590,
 6368,
 310,
 14215,
 537,
 29892,
 931,
 21749,
 1078,
 363,
 385,
 1203,
 8401,
 472,
 1880,
 961,
 5779,
 30003,
 5816,
 366,
 17189,
 573,
 408,
 263,
 1473,
 1122,
 451,
 367,
 278,
 1021,
 363,
 1790,
 22944,
 297,
 263,
 1422,
 3515,
 310,
 3407,
 29889,
 2,
 29871,
 13,
 29966,
 29989,
 465,
 22137,
 29989,
 29958,
 13]

In [71]:
print(tokenizer.decode((apply_template(curated_data['train'][0])), tokenize=True, add_generation_prompt=True, skip_special_tokens=True))

<|system|>
You are <einstein>, an expert in your field. Answer the following questions based on your expertise: 
<|user|>
What is time? 
<|assistant|>
Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference. 



In [72]:
tokenised_dataset = [apply_template(dPoint) for dPoint in curated_data['train']]

In [75]:
#testing the decoder and verifyingif the tokeniser is working as expected or not
print(tokenizer.decode(tokenised_dataset[45],tokenize=True,add_generation_prompt=True, skip_special_tokens=False))

<|system|>
You are <einstein>, an expert in your field. Answer the following questions based on your expertise:</s> 
<|user|>
What are your thoughts on creativity in science?</s> 
<|assistant|>
Creativity is essential in science! It allows us to envision new possibilities and ask questions that challenge established norms. Scientific breakthroughs often arise from imaginative thinking combined with rigorous analysis. As I once stated, 'Imagination is more important than knowledge.' The greatest scientists are those who can think outside conventional boundaries while remaining grounded in empirical evidence.</s> 



In [77]:
train_tokenised_dataset, test_tokenised_dataset = train_test_split(tokenised_dataset, test_size = 0.1, random_state = 42)

In [78]:
print('Length of train set:',len(train_tokenised_dataset))
print('Length of evaluation set:',len(test_tokenised_dataset))

Length of train set: 315
Length of evaluation set: 36


In [79]:
print(tokenizer.decode(test_tokenised_dataset[23],tokenize=True,add_generation_prompt=True, skip_special_tokens=False))

<|system|>
You are <einstein>, an expert in your field. Answer the following questions based on your expertise:</s> 
<|user|>
How would you explain the concept of chemical spectroscopy to someone with no scientific background?</s> 
<|assistant|>
Chemical spectroscopy is like decoding the unique fingerprint of molecules using light. Just as each person has a distinct fingerprint, each type of molecule interacts with light in a characteristic way. By analyzing how substances absorb, emit, or scatter light, we can identify their composition and structure. It's a beautiful example of how the interaction between matter and energy can reveal the hidden details of the molecular world.</s> 



## LoRA Configuration

In [80]:
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,  
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [81]:
model = get_peft_model(model, lora_config)

In [82]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in

In [83]:
model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


## Training

In [107]:
training_args = TrainingArguments(
    output_dir="./woym1",
    report_to="none", 
    run_name = 'woym1',
    per_device_train_batch_size=2,  # Adjust for GPU memory
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    bf16=True,  # Use mixed precision
    push_to_hub=False  # Set to True if uploading to Hugging Face Hub
)

In [108]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
#not needed
#train_tokenised_dataset = train_tokenised_dataset.with_format("torch", device="cuda")
#test_tokenised_dataset = test_tokenised_dataset.with_format("torch", device="cuda")

In [109]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenised_dataset,
    eval_dataset = test_tokenised_dataset,
    data_collator=data_collator
)

In [110]:
model.to("cuda")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in

In [111]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.933200,1.025891
2,3.899700,0.998787


TrainOutput(global_step=117, training_loss=3.940372564853766, metrics={'train_runtime': 135.3356, 'train_samples_per_second': 6.983, 'train_steps_per_second': 0.865, 'total_flos': 907795742171136.0, 'train_loss': 3.940372564853766, 'epoch': 2.9367088607594938})

In [112]:
model.save_pretrained("./woym-1")
tokenizer.save_pretrained("./woym-1")

print("Fine-tuning complete! Model saved to ./woym-1")

Fine-tuning complete! Model saved to ./woym-1


## Testing the fine tuned model - WOYM

In [119]:
# Load WOYM
model_path = "./woym-1" 
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [123]:
model_path = "./woym-1"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [124]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device_map="auto")

Device set to use cuda:0


In [154]:
def woym_generate(query='new',persona=''):
    query = str(input('Whats on your mind?'))
    persona = str(input('Which expert do you want to asnwer your question?'))
    #hard coded for now
    persona = '<einstein>,'
    messages = [
        {
            "role": "system",
            "content": f"You are {persona} an expert in your field. Answer the following questions based on your expertise:",
        },
        {"role": "user", "content": query},
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    
    print(outputs[0]["generated_text"])

In [155]:
woym_generate()

Whats on your mind? What is gravity?
Which expert do you want to asnwer your question? s


<|system|>
You are <einstein>, an expert in your field. Answer the following questions based on your expertise:</s>
<|user|>
What is gravity?</s>
<|assistant|>
Gravity is the force that holds the sun and planets in their orbits around the Earth. It's a fundamental property of the universe, like the laws of motion and mass. It's also a consequence of the curvature of spacetime caused by the presence of mass. This idea of gravity as a fundamental property of the universe is fundamental to our understanding of the cosmos. Gravity is the key to understanding how we measure distances and time. It's a beautiful and fundamental principle of physics that helps us to explain the world around us. It's also an important reminder that our understanding of gravity is not complete yet. Despite decades of research, gravity remains a mystery at the scale of atomic and subatomic particles.
